In [21]:
import os
import numpy as np
import pandas as pd
import pickle
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.gridliner import LATITUDE_FORMATTER
GeoAxes._pcolormesh_patched = Axes.pcolormesh

In [5]:
basepath = "/Users/leebardon/Dropbox/Development/stats_biogeo_2021"

In [7]:
PREDICTIONS = f"{basepath}/results/gams_output/predictions"
DARWIN = f"{basepath}/data/processed/validation_sets/plankton"
COORDS = f"{basepath}/data/processed/model_ocean_data/degrees_coords.pkl"
INNERPLOT_DATA = f"{basepath}/results/analysis_output/t_summary"

In [10]:
def get_predictions(path, *prediction_sets):
    p_sets = []
    for predictons in prediction_sets:
        with open(f"{path}/{predictons}.pkl", "rb") as handle:
            predictons_set = pickle.load(handle)
        p_sets.append(predictons_set)
    return [p_sets[i] for i in range(len(p_sets))]


def get_targets(path, *darwin_true_values):
    darwin_targets = []
    for target in darwin_true_values:
        with open(f"{path}/{target}.pkl", "rb") as handle:
            target_set = pickle.load(handle)
        darwin_targets.append(target_set)
    return [darwin_targets[i] for i in range(len(darwin_targets))]


def get_dataset(path):
    with open(f"{path}", "rb") as handle:
        dataset = pickle.load(handle)
    return dataset


def get_coords(COORDS):
    return get_dataset(COORDS)


def get_inner(basepath, *paths):
    datasets = []
    for path in paths:
        datasets.append(get_dataset(f"{basepath}/{path}"))
    return [ds for ds in datasets]


def below_cutoff_to_zero(plankton_dict):
    for group, data in plankton_dict.items():
        data[data < 1.001e-5] = 0
    return plankton_dict

In [ ]:
(
    predictions,
    predictions_r,
    predictions_f,
    predictions_rf,
    
) = get_predictions(
    
    f"{PREDICTIONS}",
    *[
        "predictions",
        "predictions_r",
        "predictions_f",
        "predictions_rf",
    ],
)

(darwin_ocean, darwin_ocean_f,) = get_targets(
    f"{DARWIN}",
    *[
        "plankton_oce",
        "plankton_oce_f",
    ],
)


coords = get_coords(COORDS)

(summary, summary_r, summary_f, summary_rf) = get_inner(
    
    f"{INNERPLOT_DATA}",
    "summary.pkl",
    "summary_r.pkl",
    "summary_f.pkl",
    "summary_rf.pkl",
)

all_plank_dicts = [
    predictions,
    predictions_r,
    predictions_f,
    predictions_rf,
    darwin_ocean,
    darwin_ocean_f,
]
[below_cutoff_to_zero(plank_dict) for plank_dict in all_plank_dicts]

In [35]:
summary[0]["Pro"]


0.11

In [25]:
def process_and_plot(data_dict, coords, filepath="", maptitle="", mtype=0):

    annual_means = {
        "Pro": 0,
        "Pico": 0,
        "Cocco": 0,
        "Diazo": 0,
        "Diatom": 0,
        "Dino": 0,
        "Zoo": 0,
    }

    means_coords = {"lon": 0, "lat": 0}

    for f_group, data in data_dict.items():
        group_df = pd.DataFrame(columns=[f"{f_group}", "lon", "lat"])
        group_df[f"{f_group}"] = data
        group_df["lon"], group_df["lat"] = (
            coords["x_deg"].values,
            coords["y_deg"].values,
        )
        means = get_annual_means(group_df, f_group)
        if mtype == 0:
            pivot_table(means, f_group, filepath, maptitle)
        else:
            annual_means[f"{f_group}"] = means[f"{f_group} annual means"].values
            means_coords["lon"] = means["lon"].values
            means_coords["lat"] = means["lat"].values

    return annual_means, means_coords



def get_annual_means(group_df, f_group):
    annual_means = (
        group_df.groupby(["lon", "lat"])[f"{f_group}"]
        .mean()
        .to_frame(name=f"{f_group} annual means")
        .reset_index()
    )
    return annual_means

In [32]:
def generate_diff_maps(darwin, gams, coords, savepath, maptitle):
    darwin_means, means_coords = process_and_plot(darwin, coords, mtype=1)
    gams_means, means_coords = process_and_plot(gams, coords, mtype=1)
    
    for f_group, data in darwin_means.items():
        diffs_df = pd.DataFrame(columns=[f"{f_group}_diffs", "lon", "lat"])
        diffs_df[f"{f_group}_diffs"] = (
            gams_means[f"{f_group}"] - darwin_means[f"{f_group}"]
        )
        diffs_df["lon"], diffs_df["lat"] = means_coords["lon"], means_coords["lat"]
        rel_diffs_da = pivot_table(diffs_df, mtype=1)
        rel_diff_maps(rel_diffs_da, savepath, f_group, maptitle)


def rel_diff_maps(rel_diffs_da, savepath, f_group, maptitle):
    lat = rel_diffs_da["lat"].data
    lon = rel_diffs_da["lon"].data
    biomass = rel_diffs_da.data
    vmin = np.percentile(biomass, 5)
    vmax = np.percentile(biomass, 95)
    plot_diff_map(lon, lat, biomass, vmin, vmax, savepath, f_group, maptitle)


def plot_diff_map(lon, lat, biomass, vmin, vmax, savepath, filename, maptitle):
    extent = [abs(vmin) if abs(vmin) > abs(vmax) else abs(vmax)]
    projection = ccrs.PlateCarree(central_longitude=180.0)
    transform = ccrs.PlateCarree()
    # cmap = "bwr"
    cmap = "Spectral_r"
    #     cmap = 'PiYG'
    #     cmap = 'PuOr'
    fc = "gray"

    fig = plt.figure(figsize=(7, 4))

    ax = plt.subplot(111, projection=projection)
    ax1 = ax.add_feature(
        cfeature.NaturalEarthFeature(
            "physical", "land", "110m", edgecolor="face", facecolor=fc
        )
    )
    ax1 = ax.pcolormesh(
        lon,
        lat,
        biomass * 100,
        cmap=cmap,
        transform=transform,
        vmin=-extent[0] * 100,
        vmax=extent[0] * 100,
        shading="gouraud",
    )
    gl = ax.gridlines(linewidth=0, draw_labels=True)
    gl.yformatter = LATITUDE_FORMATTER
    gl.top_labels, gl.left_labels, gl.right_labels, gl.bottom_labels = (
        False,
        True,
        False,
        False,
    )
    ax.coastlines(linewidth=0.3)
    ax.set_aspect("auto")
    cb = fig.colorbar(
        ax1,
        ax=ax,
        orientation="vertical",
        fraction=0.11,
        pad=0.03,
        label=" Percentage (%) ",
    )
    #     cb.set_label("Mean Difference (%)", labelpad=-1.2, fontsize=11)

    plt.title(maptitle, fontsize=13)
    plt.show()
#     plt.savefig(
#         f"{savepath}/{filename}.pdf",
#         format="pdf",
#         dpi=1200,
#         bbox_inches="tight",
#     )
#     plt.close(fig)

In [28]:

def pivot_table(means_df, f_group="", filepath="", maptitle="", mtype=0):
    annual_means_pv = means_df.pivot(index="lat", columns="lon")
    annual_means_pv = annual_means_pv.droplevel(0, axis=1)
    annual_means_da = create_datarray_object(
        annual_means_pv, f_group, filepath, maptitle, mtype
    )
    return annual_means_da


def create_datarray_object(annual_means_pv, f_group, filepath, maptitle, mtype):
    annual_means_nan_to_zero = annual_means_pv.fillna(0)
    annual_means_da = xr.DataArray(data=annual_means_nan_to_zero)
    return annual_means_da


In [34]:
generate_diff_maps(
        darwin_ocean,
        predictions,
        coords,
        "dummy_path",
        "Mean Relative Difference",
)